## Clasificación avanzada

#### SVM

El siguiente ejemplo fue tomado de la documentación oficial de scikit-learn 

https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [4]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train', 
                                  categories=categories, 
                                  shuffle=True, random_state=42)

twenty_test = fetch_20newsgroups(subset='test',
                                 categories=categories, 
                                 shuffle=True, random_state=42)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
X_train_tf = TfidfVectorizer(use_idf=False).fit_transform(twenty_train.data)
X_train_tf.shape


(2257, 35788)

In [6]:
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SVC(kernel='linear'))])

In [4]:
import numpy as np
text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(twenty_test.data)
# Calculamos accuracy:
np.mean(predicted == twenty_test.target)

0.9207723035952063

In [7]:
tf_idf = TfidfVectorizer()
X_train = tf_idf.fit_transform(twenty_train.data)
X_test = tf_idf.transform(twenty_test.data)
y_train = twenty_train.target
y_test = twenty_test.target

#### Voting

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

svc_clf = SVC(kernel='linear', probability=True) #  para soft
sgd_clf = SGDClassifier(loss='log') # para soft
voting_clf = VotingClassifier(
estimators=[('svc', svc_clf), ('sgd', sgd_clf)],voting='soft')


for clf in (svc_clf, sgd_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    
    #SVC 0.9207723035952063
#SGDClassifier 0.9247669773635153
#VotingClassifier 0.9241011984021305

SVC 0.9207723035952063
SGDClassifier 0.914780292942743
VotingClassifier 0.9300932090545939


#### Bagging

In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred)

0.7203728362183754

In [12]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=200, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [13]:
accuracy_score(y_test, y_pred)

0.7989347536617842

#### RandomForest

In [14]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8042609853528628

#### Boosting

#### AdaBoost

In [15]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
     DecisionTreeClassifier(max_depth=1), n_estimators=1500,
     algorithm="SAMME.R"
 )
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=1500)

In [16]:
y_pred = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8215712383488681

#### XgBoost

In [2]:
# pip install xgboost

from xgboost import XGBClassifier

In [10]:
xgb_clf = XGBClassifier()
xgb_clf.fit(X_train, y_train)

[09:48:54] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
y_pred = xgb_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8621837549933422

#### LightGBM

In [13]:
# conda install -c conda-forge lightgbm
import lightgbm as lgb

In [14]:
gbm = lgb.LGBMClassifier()
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)])

[1]	valid_0's multi_logloss: 1.23124
[2]	valid_0's multi_logloss: 1.11646
[3]	valid_0's multi_logloss: 1.023
[4]	valid_0's multi_logloss: 0.945858
[5]	valid_0's multi_logloss: 0.878616
[6]	valid_0's multi_logloss: 0.817269
[7]	valid_0's multi_logloss: 0.768198
[8]	valid_0's multi_logloss: 0.724117
[9]	valid_0's multi_logloss: 0.686055
[10]	valid_0's multi_logloss: 0.65002
[11]	valid_0's multi_logloss: 0.616503
[12]	valid_0's multi_logloss: 0.587707
[13]	valid_0's multi_logloss: 0.564735
[14]	valid_0's multi_logloss: 0.540743
[15]	valid_0's multi_logloss: 0.517917
[16]	valid_0's multi_logloss: 0.49996
[17]	valid_0's multi_logloss: 0.483758
[18]	valid_0's multi_logloss: 0.469801
[19]	valid_0's multi_logloss: 0.455247
[20]	valid_0's multi_logloss: 0.4451
[21]	valid_0's multi_logloss: 0.435949
[22]	valid_0's multi_logloss: 0.423442
[23]	valid_0's multi_logloss: 0.414759
[24]	valid_0's multi_logloss: 0.406735
[25]	valid_0's multi_logloss: 0.400667
[26]	valid_0's multi_logloss: 0.396664
[27]

LGBMClassifier()

In [15]:
y_pred = gbm.predict(X_test)
accuracy_score(y_test, y_pred)

0.8861517976031957

#### Stacking

In [ ]:
# pip install vecstack

In [18]:
from vecstack import StackingTransformer


estimators = [('xgb', xgb_clf),
              ('ada', ada_clf)]
              
# StackingTransformer
stack = StackingTransformer(estimators, regression=False, verbose=2)

# Fit
stack = stack.fit(X_train, y_train)

# stacked features
S_train = stack.transform(X_train)
S_test = stack.transform(X_test)


ModuleNotFoundError: No module named 'vecstack'

In [ ]:
gbm = lgb.LGBMClassifier()
gbm.fit(S_train, y_train,
        eval_set=[(S_test, y_test)])

In [ ]:
y_pred = gbm.predict(S_test)
accuracy_score(y_test, y_pred)